In [27]:
import pickle
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob
from sklearn import preprocessing

from sklearn.preprocessing import OneHotEncoder as ohc

"""sliding window에서 stepSize가 sliding 1초에 할건지 3초에할건지인지 지금은 3초로 되있음 """
def sliding_window(data,stepSize,windowSize):
    for x in range(0,len(data)-windowSize+1,stepSize):
        yield(x,data[x:x+windowSize])

"""3초씩 슬아디잉하고 여기서 데이터는 10분짜리 complex데이터임 6초 window를 3초씩 슬라이딩한 결과 (199,180)짜리 test set이 나옴
여기서 180은 simpled activity에서 feature가 5개이고, 36개 acc, gyro list에서 피처뽑아서 5*180
그걸 바로 clf로 인퍼런스하는게 출력"""
def pred_window(clf,data):
    test_set=[]
    for(x,window) in sliding_window(data,3,6): 
        wind=window.reshape(180)
        test_set.append(wind)
    #print(np.shape(test_set))
    return clf.predict(test_set)

"""pred_window랑 똑같은데 출력만 다름 확률뽑을때 predict하기 전 test_set이 필요해서 """
def test_set(data):
    test_set=[]
    for(x,window) in sliding_window(data,3,6): 
        wind=window.reshape(180)
        test_set.append(wind)
    return (test_set)

def stdmean(window):
    RMS_window=[]
    for i in range(0,180,6):
        RMS_window.append(float(window[i]))
    RMS_window=np.array(RMS_window)
    return RMS_window.std(axis=-1)

#filename queue
filename_queue=glob.glob('./featuredata/compfeature[1-2].csv')
data=list()
context_label=list()
context_label_count=np.zeros(4)

for filename in filename_queue:
    print(filename)
    file=open(filename, newline='')
    reader=csv.reader(file)
    #context=reader[seq(1, nrow(reader), 2),]
    #data = reader[seq(1, nrow(reader), 2),]
    i=0
    array=list()


    for row in reader:
        i+=1
        if(int(i)%2==1):
            context_label.append(row)
            context_label_count[int(row[0])]+=1
        else:
            array=np.array(row)
            array=array.reshape(600,6,5)
            data.append(array)


./featuredata/compfeature2.csv
./featuredata/compfeature1.csv


In [28]:
# load svm
filename='./svm.sav'
svm = pickle.load(open(filename, 'rb'))

# load knn
filename='./knn.sav'
knn = pickle.load(open(filename, 'rb'))

# load rf
filename='./rf.sav'
rf = pickle.load(open(filename, 'rb'))

# load ETC
filename='./ETC.sav'
ETC = pickle.load(open(filename, 'rb'))

In [29]:
test_Set=list()
stdStd=[]
stdMean=[]
pred=[]
pred2=[]
pred3=[]
blacklist=np.array([0,8])
test_pred_proba=[]
for x in data:
    test_Set.append(test_set(x))
#print(np.shape(test_set1))
test_Set=np.array(test_Set)
#print(np.shape(test_set1[0]))
#print((test_set1[0][0]))

###여기는 test set에서 stdmean함수로 계산한 걸 stdMean에다가 저장했어요. stdMean은 길이가 pred랑 똑같습니다.
for x in test_Set:
    for i in x:
        stdMean.append((stdmean(i)))
stdMean=np.array(stdMean)
np.set_printoptions(precision=2)
print('stdMean25: ',np.percentile(stdMean,25))

for x in test_Set:
    test_pred_proba.append(ETC.predict_proba((x))) 

for i in test_pred_proba:
    for j in i:
        if np.max(j)<0.2: #확률 0.3미만이면 idle
            pred.append(10)
        else:
            if np.argmax(j) in [8,0]:
                #print('label : ',np.argmax(j), 'probability : ',np.max(j))
                if(np.max(j)<0.25):
                    pred.append(10)
                else:
                    pred.append(np.argmax(j))
            else:
                if np.argmax(j)==3:
                #print('label : ',np.argmax(j), 'probability : ',np.max(j))
                    if(np.max(j)<0.24):
                        pred.append(10)
                    else:
                        pred.append(3)
                else:
                    pred.append(np.argmax(j))

"""이부분에서  stdMean이 일정 이하인 애들은 pred에서 버렸어요"""
print(len(pred))
print(len(stdMean))
for i in range(len(pred)):
    if(float(stdMean[i])>=0.5):
        pred2.append(pred[i])
    else:
        pred2.append(10)

#pred.reshape(-1,199)
print(np.shape(test_Set))
print(np.shape(pred))
print(np.shape(stdMean))
print((stdMean[0]))
print(np.shape(pred2))



"""여긴 그냥 사이즈 맞춰줄려고 199길이 리스트에다가 +'n'추가할거임"""

for i in range(int(len(pred2)/199)):
    pred3.append(pred2[199*i:199*i+199])
for i in pred3:
    i.append(10)
pred3=np.array(pred3)
print(np.shape(pred3))

#print('pred3 : ',np.shape(pred3))


pred3=pred3.reshape(len(data),10,20)

#print(context_label)
print(list(pred3))
"""21,199,10은 test_pred_proba의 shape인데 이게 뭐나면 라벨이 10개여서 마지막에 10이고 각각의 확률 나오는거임
21은 현재까지 모인 Context 데이터 개수(쏘리 말잘못했음 전화할때) 199는 10분을 window Sliding 하면 199나옴"""

stdMean25:  0.5744166254992193
10348
10348
(52, 199, 180)
(10348,)
(10348,)
2.8178884936966533
(10348,)
(52, 200)
[array([[10, 10, 10, 10, 10, 10, 10,  0, 10, 10, 10,  0, 10, 10, 10,  3,
        10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10,  3, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         0, 10, 10, 10],
       [10, 10, 10, 10, 10,  0, 10, 10, 10, 10, 10, 10, 10, 10,  3, 10,
        10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  0, 10, 10, 10, 10,
        10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10],
       [10, 10, 10,  0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10,  3, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10,  0, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10],
       [10, 10, 10,  3, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10],
       [10, 

'21,199,10은 test_pred_proba의 shape인데 이게 뭐나면 라벨이 10개여서 마지막에 10이고 각각의 확률 나오는거임\n21은 현재까지 모인 Context 데이터 개수(쏘리 말잘못했음 전화할때) 199는 10분을 window Sliding 하면 199나옴'

라벨-액티비티
0 얼굴만지기
1 필기하기
2 건배하기
3 마시기1
4 휴지뽑기
5 마시기2
6 숟가락
7 젓가락 
8 포크
9 젓기
16 idle

빠진 액티비티 키보드, 마우스, 손들기, 사진찍기, 술 따르기, 스마트폰 타이핑

In [30]:
### Context prediction 용 데이터 만들기 길이 (라벨10개+idle1개)11*(10분)10 = 110
context_data=[]
for context in pred3:
    context_=[]
    for minute in context:
        activity_count=np.zeros(11)
        for activity in minute:
            activity_count[activity]+=1
        context_.append(activity_count)
        context=list(context)
    context_data.append(context_)
context_data=np.array(context_data)
context_data=context_data.reshape(len(context_data),110)


In [31]:
print(np.shape(context_data))
print(np.shape(context_label))
print(context_label)

(52, 110)
(52, 1)
[['1'], ['1'], ['1'], ['1'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['2'], ['2'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['2'], ['2'], ['1'], ['1'], ['0'], ['0'], ['0'], ['0'], ['2'], ['2'], ['2'], ['2'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['1'], ['1'], ['0'], ['0'], ['1'], ['0'], ['0'], ['3'], ['2'], ['0'], ['0'], ['3']]


In [32]:
X=context_data

y=context_label


In [33]:
### 여기서부터 context prediction 시작
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV

clf=RandomForestClassifier()
clf.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score 
scores=cross_val_score(clf,X,y,cv=3,scoring='accuracy')
print(scores.mean())

0.9004629629629629


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_va

In [35]:
from sklearn.model_selection import GridSearchCV

#X=preprocessing.minmax_scale(training_set)
#X=preprocessing.robust_scale(training_set)
#X=preprocessing.quantile_transform(training_set)

RF=RandomForestClassifier()
param_grid={
 'n_estimators':[50,80,100,120,200],
    'max_features':['auto','sqrt','log2'],
    'max_depth':[10,11,9,None]
    #'criterion':['gini','entropy']
}
grid=GridSearchCV(estimator=RF,param_grid=param_grid,cv=3,scoring='accuracy')

In [36]:
grid.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_va

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_s

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_s

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_s

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_s

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_s

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_s

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [37]:
grid.cv_results_

{'mean_fit_time': array([0.03, 0.04, 0.05, 0.06, 0.1 , 0.03, 0.05, 0.09, 0.06, 0.09, 0.02,
        0.04, 0.08, 0.08, 0.09, 0.02, 0.03, 0.05, 0.06, 0.1 , 0.02, 0.04,
        0.05, 0.06, 0.09, 0.02, 0.04, 0.05, 0.05, 0.09, 0.02, 0.04, 0.05,
        0.05, 0.2 , 0.02, 0.04, 0.05, 0.05, 0.09, 0.02, 0.03, 0.05, 0.05,
        0.15, 0.04, 0.04, 0.05, 0.05, 0.09, 0.02, 0.04, 0.05, 0.05, 0.09,
        0.02, 0.04, 0.05, 0.05, 0.09]),
 'std_fit_time': array([5.47e-03, 7.23e-04, 3.38e-03, 1.54e-03, 3.91e-03, 4.22e-03,
        1.75e-02, 5.32e-02, 3.26e-03, 4.05e-03, 2.83e-04, 2.59e-03,
        3.44e-02, 4.40e-02, 1.98e-03, 6.10e-04, 1.43e-05, 1.16e-03,
        4.45e-03, 7.67e-03, 5.26e-04, 1.24e-03, 2.63e-03, 3.59e-03,
        2.60e-03, 1.24e-03, 3.71e-03, 3.64e-03, 2.62e-03, 5.33e-03,
        5.80e-04, 2.50e-03, 1.62e-03, 1.46e-03, 7.56e-02, 1.74e-03,
        1.24e-04, 3.30e-03, 3.05e-03, 3.24e-03, 7.29e-04, 1.01e-04,
        3.82e-03, 1.98e-03, 7.55e-02, 1.76e-02, 4.17e-03, 2.12e-03,
        1.43e

In [38]:
grid.best_params_

{'max_depth': 10, 'max_features': 'log2', 'n_estimators': 50}

In [39]:
from joblib import dump, load
complex_RF=RandomForestClassifier(max_depth=10,max_features='log2',n_estimators=50)
complex_RF.fit(X,y)
filename='./complex_rf.sav'
pickle.dump(complex_RF,open(filename,'wb'))
dump(complex_RF,'complex_rf.joblib')

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


['complex_rf.joblib']

In [40]:
X=(context_data)
y=context_label


In [41]:
clf1=ExtraTreesClassifier()
clf1.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [42]:

scores=cross_val_score(clf1,X,y,cv=3,scoring='accuracy')
print(scores.mean())

0.9004629629629629


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_va

In [43]:
param_grid={
 'n_estimators':[100,200,300],
   'max_features':['auto','sqrt','log2'],
    'max_depth':[10,14,None]
    #'criterion':['gini','entropy']
}

grid=GridSearchCV(estimator=ETC,param_grid=param_grid,cv=3,scoring='accuracy')

In [45]:
grid.fit(X,y)


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_va

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_s

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_s

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None,
                                            criterion='gini', max_depth=30,
                                            max_features='sqrt',
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=300, n_jobs=None,
                                            oob_score=False, random_state=None,
                                            verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10, 14, None]

In [46]:
grid.cv_results_

{'mean_fit_time': array([0.08, 0.08, 0.12, 0.04, 0.1 , 0.15, 0.04, 0.08, 0.13, 0.1 , 0.08,
        0.13, 0.09, 0.08, 0.12, 0.04, 0.08, 0.19, 0.04, 0.13, 0.12, 0.09,
        0.09, 0.17, 0.04, 0.14, 0.12]),
 'std_fit_time': array([0.03, 0.  , 0.01, 0.  , 0.02, 0.03, 0.  , 0.  , 0.01, 0.01, 0.  ,
        0.01, 0.01, 0.  , 0.  , 0.  , 0.  , 0.1 , 0.  , 0.06, 0.  , 0.02,
        0.01, 0.07, 0.  , 0.06, 0.  ]),
 'mean_score_time': array([0.  , 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01, 0.01,
        0.02, 0.01, 0.01, 0.01, 0.  , 0.01, 0.02, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.  , 0.02, 0.01]),
 'std_score_time': array([8.15e-05, 1.50e-04, 1.73e-04, 1.37e-03, 4.17e-03, 8.24e-05,
        1.87e-04, 2.14e-03, 2.21e-03, 4.35e-03, 1.24e-04, 4.04e-03,
        2.80e-03, 9.38e-05, 4.48e-05, 5.59e-05, 1.30e-03, 5.04e-03,
        2.73e-03, 4.00e-03, 4.31e-04, 1.29e-03, 5.95e-03, 3.78e-05,
        9.46e-06, 4.48e-03, 7.48e-05]),
 'param_max_depth': masked_array(data=[10, 10, 10, 10,

In [47]:
grid.best_params_

{'max_depth': 14, 'max_features': 'auto', 'n_estimators': 200}

In [48]:
from joblib import dump, load
ETC=ExtraTreesClassifier(max_depth=14,n_estimators=200,max_features='auto')
ETC.fit(X,y)
filename='./complex_ETC.sav'
pickle.dump(clf,open(filename,'wb'))
dump(clf,'complex_ETC.joblib')

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


['complex_ETC.joblib']

In [49]:
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(complex_RF, X, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:867: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:867: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_v

[[32  0  0  0]
 [ 2  7  0  0]
 [ 0  0  9  0]
 [ 0  0  0  2]]


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:867: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:867: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:867: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
